In [25]:
import math
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler 
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [115]:
#load dataset
#taking alphabet as an exmaple to test out prediction
StockData = pd.read_csv("/content/drive/MyDrive/Datasets/ALPH.csv")

In [116]:
StockData["Date"] = pd.to_datetime(StockData["Date"])

In [ ]:
FullData=StockData[['Adj Close']].values
print(FullData[0:5])
from sklearn.preprocessing import StandardScaler, MinMaxScaler

sc=MinMaxScaler()
 
DataScaler = sc.fit(FullData)
X=DataScaler.transform(FullData)
X[0:5]

# Predicting for future 5 days

In [ ]:
print('Original Prices')
print(FullData[-10:])

X=X.reshape(X.shape[0],)
print('Scaled Prices')
print(X[-10:])

In [119]:
# split into samples
X_samples = list()
y_samples = list()
 
NumerOfRows = len(X)
TimeSteps=10
FutureTimeSteps=5
 
# Iterate thru the values to create combinations
for i in range(TimeSteps , NumerOfRows-FutureTimeSteps , 1):
    x_sample = X[i-TimeSteps:i]
    y_sample = X[i:i+FutureTimeSteps]
    X_samples.append(x_sample)
    y_samples.append(y_sample)

X_data=np.array(X_samples)
X_data=X_data.reshape(X_data.shape[0],X_data.shape[1], 1) 
y_data=np.array(y_samples)

In [120]:
TestingRecords=5

X_train=X_data[:-TestingRecords]
X_test=X_data[-TestingRecords:]
y_train=y_data[:-TestingRecords]
y_test=y_data[-TestingRecords:]

In [ ]:
for inp, out in zip(X_train[0:2], y_train[0:2]):
    print(inp)
    print('====>')
    print(out)
    print('#'*20)

In [ ]:
TimeSteps=X_train.shape[1]
TotalFeatures=X_train.shape[2]
print("Number of TimeSteps:", TimeSteps)
print("Number of Features:", TotalFeatures)

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 200, activation = 'relu', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))
regressor.add(LSTM(units = 5, activation = 'relu', return_sequences=False ))
regressor.add(Dense(units = FutureTimeSteps))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

StartTime=time.time()
 
# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, batch_size = 32, epochs = 100)
 
EndTime=time.time()
print("Time --> ", round((EndTime-StartTime)/60), 'Minutes')

In [ ]:
predicted_Price = regressor.predict(X_test)
predicted_Price = DataScaler.inverse_transform(predicted_Price)
print(predicted_Price)

orig=y_test
orig=DataScaler.inverse_transform(y_test)
print(orig)

In [ ]:
Last10DaysPrices = np.array([2609.51, 2534.8, 2593.21, 2673.81, 2692.01, 2736.03, 2729.57, 2805.55, 2770.07, 2826.24])
Last10DaysPrices=Last10DaysPrices.reshape(-1, 1)

X_test=DataScaler.transform(Last10DaysPrices)
 
NumberofSamples=1
TimeSteps=X_test.shape[0]
NumberofFeatures=X_test.shape[1]
X_test=X_test.reshape(NumberofSamples,TimeSteps,NumberofFeatures)

Next5DaysPrice = regressor.predict(X_test)

Next5DaysPrice = DataScaler.inverse_transform(Next5DaysPrice)
for line in Next5DaysPrice:
    print("Predicted values --> ",*line)